# Arcregeneráció rossz minőségű képekből

Szükséges modulok importálása

In [8]:
import sys
sys.path.append('c:\\users\\ifjto\\appdata\\local\\programs\\python\\python37\\lib\\site-packages')
import os
import torch
import torch.cuda
import torchvision
import numpy as np
import torch.nn as nn
from torch.utils.data import *
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
import cv2
import os
import time
from IPython.display import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import random
import face_recognition

Dataset objektum az autoenkóder adatbázisának.

In [9]:
class AutoEncoderDataset(Dataset):
    def __init__(self, pictures):
        self.pictures = pictures
        self.counter = 0
    def __getitem__(self, index):
        self.counter += 1
        if self.counter > len(self.pictures)-1:
            self.counter = 0
        return self.pictures[self.counter][1], self.pictures[self.counter][0]
    def __len__(self):
        return len(self.pictures)

Dataset objektum a Diszkriminátor számára. A képeket ez is beolvassa megint, ami felesleges munka, azonban ennyi adatnál még nem túl sok pluszidő a lefutása.

In [10]:
class DiscriminatorDataset(Dataset):
    def __init__(self, TRAIN_SIZE):
        self.max_counter = TRAIN_SIZE
        self.pictures = []
        for pic in os.listdir("faces"):
            image = cv2.imread("faces/"+pic)
            image = torch.Tensor(image).permute(2, 1, 0)
            image = (2/255) * image - torch.ones(3, 128, 128)
            self.pictures.append(image)
        self.fakes = []
        self.counter = 0
    def __getitem__(self, index):
        self.counter += 1
        if self.counter > self.max_counter-1:
            self.counter = 0
        if self.counter % 2:
            return self.pictures[self.counter//2], 0
        return self.fakes[self.counter//2], 1
    def __len__(self):
        return len(self.pictures)+len(self.fakes)

Segédfüggvény külső forrásból a beolvasás állapotának kijelzésére. Ez azért fontos, mert nagy mennyiségű adat áll rendelkezésre, így a beolvasás gyakran lassú, és jó tudni, hogy halad-e, illetve mennyi van hátra ebből a lépésből.

In [11]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

Pixelesített képeket létrehozó függvény. Itt fontos, hogy milyen interpolációt választunk. Mivel a háló erre is rá tud tanulni, ezért a legegyszerűbb interpolációt választjuk, hogy ne vezessük vele félre. Beolvasáskor normáljuk is a képeket a numerikus konvergencia támogatásáért.
Használjunk adat augmentációt a pontosság és az adatmennyiség növelésére. Augmentációnak a tükrözést választottam, mivel ez mindenképp megfelelő képet ad eredményül.

In [12]:
def get_record(pic, augmentation=False):
    image = cv2.imread(pic)
    image2 = image[:, ::-1, :].copy()
    
    #real images
    real = torch.Tensor(image).permute(2, 1, 0)
    real = (2/255) * real - torch.ones(3, 128, 128)
        
    # pixelised image
    pixel = cv2.resize(image, (32, 32), interpolation=cv2.INTER_LINEAR)
    pixel = torch.Tensor(pixel).permute(2, 1, 0)
    pixel = (2/255) * pixel - torch.ones(3, 32, 32)
    
    if augmentation:
        real2 = torch.Tensor(image2).permute(2, 1, 0)
        real2 = (2/255) * real2 - torch.ones(3, 128, 128)
        pixel2 = cv2.resize(image2, (32, 32), interpolation=cv2.INTER_LINEAR)
        pixel2 = torch.Tensor(pixel2).permute(2, 1, 0)
        pixel2 = (2/255) * pixel2 - torch.ones(3, 32, 32)
        return (real, pixel), (real2, pixel2)
    return (real, pixel)

Olvassuk be a képeket egy Dataset tárolóobjektumba, és pixelesítsük. Osszuk szét az adatot tanító-, teszt- és validációs adatbázisok között, 70:15:15 arányban. Használjunk az ellenőrzés kedvéért egy meghatározott random magot. Rendezzük a képeket kötegekbe.

In [15]:
# the maximum number of pictures included in each epoch
# must be defined before reading in the dataset
#TRAIN_SIZE = 5000

def get_data(batch_size = 32, TRAIN_SIZE=5000):
    np.random.seed(42)
    torch.manual_seed(42)

    # reading pictures
    files = os.listdir("faces")
    np.random.shuffle(files)
    train_pictures = []
    for pic in log_progress(files[:int(len(files)*0.7)], every=100):
        p1, p2 = get_record("faces/"+pic, True)
        train_pictures.append(p1)
        train_pictures.append(p2)
    test_pictures = []
    for pic in log_progress(files[int(len(files)*0.7):int(len(files)*0.85)], every=100):
        p = get_record("faces/"+pic)
        test_pictures.append(p)
    val_pictures = []
    for pic in log_progress(files[int(len(files)*0.85):], every=100):
        p = get_record("faces/"+pic)
        val_pictures.append(p)
    
    train_dataset = AutoEncoderDataset(train_pictures)
    test_dataset = AutoEncoderDataset(test_pictures)
    validation_dataset = AutoEncoderDataset(val_pictures)

    size = min(TRAIN_SIZE, len(train_pictures))
    print("Epoch size:", size)
    sampler = SubsetRandomSampler(range(size))
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
    validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)
    
    discriminator_dataset = DiscriminatorDataset(TRAIN_SIZE)
    sampler = SubsetRandomSampler(range(size))
    disc_loader = torch.utils.data.DataLoader(discriminator_dataset, batch_size=batch_size, sampler=sampler)
    
    return trainloader, testloader, validationloader, discriminator_dataset, disc_loader

Autoencoder osztály az arcok generálására. A normál autoencoderektől eltér abban, hogy nem szimmetrikus, hiszen rosszabb minőségből jobbat kell előállítania, így a generált kép szükségszerűen nagyobb, mint az eredeti.

In [6]:
class Autoencoder(nn.Module):
    def __init__(self, nFeat, dropout=0.1):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, nFeat, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(nFeat, nFeat//2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(nFeat//2, nFeat//4, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(nFeat//4, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=dropout),
            nn.MaxPool2d(2, 2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, padding=1)
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = torch.sigmoid(x)
        return x

Diszkriminátor osztály. Ennek feladata hasonlít a GAN hálók diszkriminátorára, vagyis azt mondja meg, hogy mennyire hasonlít egy kép egy igazi emberi arcra, és mennyire egy generáltra. Ez az eredmény az autoencoder loss függvényében tölt be szerepet.

In [7]:
class Discriminator(nn.Module):
    def __init__(self, dropout=0.2):
        super(Discriminator, self).__init__()
        ndf = 32
        self.main = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(3, ndf, 4, 4, 1, bias=False),
            nn.Dropout2d(p=dropout),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.Dropout2d(p=dropout),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.Dropout2d(p=dropout),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.main(x).squeeze()
        return x

Saját loss függvény az autoencoder számára. A hiba számítása az alábbi képlettel írható fel:
### (output - target)^2 * (rate + (1 - rate) * D(output))

, ahol D(output) a diszkriminátor háló eredménye 0 és 1 közé normalizálva. Ez garantálja, hogy a háló ne csak az eredeti képre hasonlító elmosott képet generáljon, hanem emberi arcra is - még ha az eltér a visszaállítandó képtől is.

In [8]:
def auto_loss(output, target, discriminator, ratio=0.5):
    d = discriminator(output)
    loss = 0
    for i in range(len(d)):
        loss += torch.mean((output[i]-target[i])**2)*(d[i]*(1-ratio)+ratio)
    return loss

Tanító függvény. Először az autoencodert tanítjuk, majd annak a frissen generált képeivel tanítjuk a diszkriminátor hálót is.

In [9]:
def train(autoencoder, discriminator, trainloader, discriminator_dataset, disc_loader,
          NUM_EPOCH, ratio, steps_without_disc, TRAIN_SIZE, AUTO_LR, DISC_LR, wd):
    
    train_loss = []
    disc_loss = [0 for i in range(5)]
    
    # creating components
    mse_loss = nn.MSELoss()
    auto_optimizer = optim.Adam(autoencoder.parameters(), lr=AUTO_LR)
    auto_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(auto_optimizer, NUM_EPOCH)
    disc_optimizer = optim.Adam(discriminator.parameters(), lr=DISC_LR, weight_decay=wd)
    disc_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(disc_optimizer, NUM_EPOCH)
    disc_criterion = nn.BCELoss()
    if torch.cuda.is_available():
        mse_loss = mse_loss.cuda()
        disc_criterion = disc_criterion.cuda()
    
    # training the network
    autoencoder.train(mode=True)
    for epoch in range(NUM_EPOCH):
        # training the autoencoder
        
        generated_images = []
        running_loss = 0.0
        for data in trainloader:
            pixelised, real = data
            if torch.cuda.is_available():
                pixelised = pixelised.cuda()
                real = real.cuda()
            auto_optimizer.zero_grad()
            outputs = autoencoder(pixelised)
            if epoch < steps_without_disc+1:
                loss = mse_loss(outputs, real)
            else:
                loss = auto_loss(outputs, real, discriminator, ratio)
            loss.backward()
            auto_optimizer.step()
            
            running_loss += loss.item()
            for i in outputs:
                generated_images.append(i.detach().cpu())
        
        loss = running_loss / len(trainloader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.6f}'.format(epoch+1, NUM_EPOCH, loss))
        auto_scheduler.step()
        
        if epoch < steps_without_disc:
            continue
        
        #updating the discriminator dataset with the freshly generated faces
        generated_images = random.sample(generated_images, int(TRAIN_SIZE/2))
        discriminator_dataset.fakes = generated_images
        
        # training the discriminator
        running_loss = 0.0
        total = 0
        correct = 0
        for data in disc_loader:
            images, labels = data
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()
            disc_optimizer.zero_grad()
            outputs = discriminator(images)
            loss = disc_criterion(outputs, labels.float())
            loss.backward()
            disc_optimizer.step()
            running_loss += loss.item()
            
            # calculating accuracy for discriminator
            predictions = torch.round(outputs.data.squeeze())
            correct += (predictions == labels).sum().item()
            total += images.size(0)
        print('\tDiscriminator Accuracy: {:.6f}'.format(correct/total))
        
        loss = running_loss / len(disc_loader)
        disc_loss.append(loss)
        print('\tDiscriminator Loss: {:.6f}'.format(loss))
        disc_scheduler.step()
    
    for i in range(steps_without_disc):
        disc_loss[i] = disc_loss[steps_without_disc]
    return train_loss, disc_loss

Tesztelő függvény, ami a validációra is alkalmas. Fontos, hogy elkülönüljön a teszt és a validáció, hiszen amikor teszteredményekre optimalizáljuk a hiperparamétereket, akkor ezzel be is szennyezzük a tesztadatokat - előfordulhat, hogy a paraméterek állítgatásával manuálisan megtalálunk egy véletlenül nagyon jó eredményt, ami azonban csak a tesztadatok sajátos eloszlása miatt ilyen.

In [1]:
def get_result(autoencoder, loader):
    generated_images = []
    mse_loss = nn.MSELoss()
    if torch.cuda.is_available():
        mse_loss = mse_loss.cuda()
    
    # getting the MSE loss
    autoencoder.train(mode=False)
    autoencoder_loss = 0.0
    for data in loader:
        pixelised, real = data
        if torch.cuda.is_available():
            pixelised = pixelised.cuda()
            real = real.cuda()
        outputs = autoencoder(pixelised)
        autoencoder_loss += mse_loss(outputs, real).item()
        for i in range(len(outputs)):
            generated = outputs[i].detach().cpu()
            generated = torch.round((generated + torch.ones(3, 128, 128))*0.5*255).type(torch.ByteTensor)
            generated = generated.numpy().transpose(2, 1, 0)
            original = pixelised[i].cpu()
            original = torch.round((original + torch.ones(3, 32, 32))*0.5*255).type(torch.ByteTensor)
            original = original.numpy().transpose(2, 1, 0)
            generated_images.append( (generated, original) )
    autoencoder_loss = (1 - torch.sigmoid( torch.tensor(autoencoder_loss / len(loader)) ).item())*2
    
    # getting the reality score
    reality_score = 0.0
    for i in generated_images:
        face_encoding0 = face_recognition.face_encodings(i[0])
        if not face_encoding0:
            continue
        face_encoding1 = face_recognition.face_encodings(i[1])
        if not face_encoding1:
            continue
        results = face_recognition.face_distance([face_encoding1[0]], face_encoding0[0])
        reality_score += results[0]
    reality_score /= len(generated_images)
    print(reality_score)
    
    return autoencoder_loss, reality_score

Egyéb segédfüggvények.

In [11]:
def print_tensor(t, size=128):
    t = t[0].cpu().detach()
    t = (t + torch.ones(3, size, size))*0.5
    t = t.numpy().transpose(2, 1, 0)
    plt.imshow(cv2.cvtColor(t, cv2.COLOR_BGR2RGB))
    plt.show()
    
def redraw(net, pic):
    if torch.cuda.is_available():
        net = net.cuda()
        pic = pic.cuda()
    net.train(mode=False)
    out = net(pic)
    return out
    #print(torch.max(out), torch.min(out))